In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.listdir('/kaggle/input/histopathologic-cancer-detection')

In [ ]:
os.listdir('/kaggle/input/histopathologic-cancer-detection/train')[0]

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from fastai import *
from fastai.vision import *

In [ ]:
labels = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
labels.head()

In [ ]:
sns.countplot(x='label',data=labels)
plt.show()

A positive label means that there is at least one pixel of tumor tissue in the center region (32 x 32px) of the image.

In [ ]:
path=Path('/kaggle/input/histopathologic-cancer-detection/')
tfms = get_transforms(do_flip=True,flip_vert=True, max_warp=0, max_rotate=10, max_lighting=0.05)

In [ ]:
path

In [ ]:
data = ImageDataBunch.from_csv(path, 
                             csv_labels='train_labels.csv', 
                             folder='train', 
                             suffix='.tif',
                             num_workers=2,
                             ds_tfms=tfms,
                             bs=64,
                             size=72,
                             test=path/'test').normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.c

In [ ]:
data

In [ ]:
data.show_batch(rows=3,figsize=(8,10))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

In [ ]:
learn= create_cnn(data, models.resnet34, metrics=[accuracy],model_dir='/kaggle/working/')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
#wd = weight decay
learn.fit_one_cycle(6, max_lr=(1e-4, 1e-3, 1e-2), wd=(1e-4, 1e-4, 1e-1))

Unfreeze the layers- weights of all the fozen layers will be updated

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(15, slice(1e-5,1e-4,1e-3))

In [ ]:
learn.save('stage-1') 

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,8))

**Validation Accuracy**

In [ ]:
preds,y = learn.get_preds(DatasetType.Valid)

**Accuracy**

In [ ]:
accuracy(preds,y)

**Predictions on the Test Data**

In [ ]:
label,y=learn.get_preds(DatasetType.Test)

In [ ]:
sub = pd.read_csv(f'{path}/sample_submission.csv').set_index('id')
sub.head()

In [ ]:
data.test_ds.items[0]

In [ ]:
names=np.vectorize(lambda img_name: str(img_name).split('/')[-1][:-4]) 


In [ ]:
file_names= names(data.test_ds.items).astype(str)

In [ ]:
label.numpy()[:,1]

In [ ]:
sub.loc[file_names,'label']=label.numpy()[:,1]
sub.to_csv(f'submission_resnet34.csv')

In [ ]:
sub.head()

In [ ]:
predictions = []
for i in label:
    predictions.append(i.argmax().item())

In [ ]:
sns.countplot(predictions)

In [ ]:
learn.show_results()

**Visualize Predictions made by the resnet34 Model**

In [ ]:
_,axs = plt.subplots(3,5,figsize=(11,8))
for i,ax in enumerate(axs.flatten()): 
  img = data.test_ds[i][0]
  img.show(ax=ax,y=learn.predict(img)[0])